# IMPORTING REQUIRED LIBRARIES

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np
import pandas as pd
import re
import nltk
from collections import Counter
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min


# READING THE DATA

In [ ]:
data=pd.read_csv(r"/content/IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# EDA

In [ ]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
data.isnull()

,review,sentiment
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
49995,False,False
49996,False,False
49997,False,False
49998,False,False


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


# PREPROCESSING


In [ ]:
def remove_tags(string):
    removelist = ""
    result = re.sub('','',string)          #remove HTML tags
    result = re.sub('https://.*','',result)   #remove URLs
    result = result.lower()
    result = result.replace('<br />', '')
    return result
data['review']=data['review'].apply(lambda cw : remove_tags(cw))

In [ ]:
reviews=''.join(data[data['sentiment']=='positive']['review'])
len(reviews)

32532230

In [ ]:
reviews[:1467]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

# SUMMARIZATION

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

def get_embeddings(text):
    # Check if the text is not empty or None
    if text:
        marked_text = "[CLS] " + text + " [SEP]"
        tokenized_text = tokenizer.tokenize(marked_text)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(tokenized_text)

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        # Run the text through BERT, and collect all of the hidden states produced from all 12 layers.
        with torch.no_grad():
            outputs = model(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]

        # token_vecs is a tensor with shape [22 x 768]
        token_vecs = hidden_states[-2][0]

        # Calculate the average of all 22 token vectors.
        sentence_embedding = torch.mean(token_vecs, dim=0)

        return sentence_embedding.numpy()
    else:
        # Return a zero vector for empty sentences
        return np.zeros((768,))

# Add a count function to count the words in the text
def count_words(text):
    words = text.split()
    return len(words)

# Get user-defined input for the text
text = input("Enter your text: ")

def summarize(text):
    sentences = text.split('.')
    embeddings = [get_embeddings(sentence) for sentence in sentences]
    n_clusters = int(np.ceil(len(embeddings) ** 0.5))
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans = kmeans.fit(embeddings)
    avg = []
    closest = []
    for j in range(n_clusters):
        idx = np.where(kmeans.labels_ == j)[0]
        avg.append(np.mean(idx))
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
    ordering = sorted(range(n_clusters), key=lambda k: avg[k])
    summary = ' '.join([sentences[closest[idx]] for idx in ordering])
    return summary


Enter your text: Multi-head attention The above visualization shows one attention mechanism within the model. BERT actually learns multiple attention mechanisms, called heads, which operate in parallel to one another. As we’ll see shortly, multi-head attention enables the model to capture a broader range of relationships between words than would be possible with a single attention mechanism.  BERT also stacks multiple layers of attention, each of which operates on the output of the layer that came before. Through this repeated composition of word embeddings, BERT is able to form very rich representations as it gets to the deepest layers of the model.  Because the attention heads do not share parameters, each head learns a unique attention pattern. The version of BERT that we consider here — BERT Base — has 12 layers and 12 heads, resulting in a total of 12 x 12 = 144 distinct attention mechanisms. We can visualize attention in all of the heads at once, using the model view (available i

# Outputs

In [ ]:
print("Original text:\n")
print(text)
print("Word count:", count_words(text), "\n")

Original text:

Multi-head attention The above visualization shows one attention mechanism within the model. BERT actually learns multiple attention mechanisms, called heads, which operate in parallel to one another. As we’ll see shortly, multi-head attention enables the model to capture a broader range of relationships between words than would be possible with a single attention mechanism.  BERT also stacks multiple layers of attention, each of which operates on the output of the layer that came before. Through this repeated composition of word embeddings, BERT is able to form very rich representations as it gets to the deepest layers of the model.  Because the attention heads do not share parameters, each head learns a unique attention pattern. The version of BERT that we consider here — BERT Base — has 12 layers and 12 heads, resulting in a total of 12 x 12 = 144 distinct attention mechanisms. We can visualize attention in all of the heads at once, using the model view (available in

In [ ]:
# Print the tokenized text
print("Tokenized text:\n")
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)
print("\n")

Tokenized text:

['multi', '-', 'head', 'attention', 'the', 'above', 'visual', '##ization', 'shows', 'one', 'attention', 'mechanism', 'within', 'the', 'model', '.', 'bert', 'actually', 'learns', 'multiple', 'attention', 'mechanisms', ',', 'called', 'heads', ',', 'which', 'operate', 'in', 'parallel', 'to', 'one', 'another', '.', 'as', 'we', '’', 'll', 'see', 'shortly', ',', 'multi', '-', 'head', 'attention', 'enables', 'the', 'model', 'to', 'capture', 'a', 'broader', 'range', 'of', 'relationships', 'between', 'words', 'than', 'would', 'be', 'possible', 'with', 'a', 'single', 'attention', 'mechanism', '.', 'bert', 'also', 'stacks', 'multiple', 'layers', 'of', 'attention', ',', 'each', 'of', 'which', 'operates', 'on', 'the', 'output', 'of', 'the', 'layer', 'that', 'came', 'before', '.', 'through', 'this', 'repeated', 'composition', 'of', 'word', 'em', '##bed', '##ding', '##s', ',', 'bert', 'is', 'able', 'to', 'form', 'very', 'rich', 'representations', 'as', 'it', 'gets', 'to', 'the', 'dee

In [ ]:
print("Summary:\n")
print(summarize(text))
print("Word count:", count_words(summarize(text)), "\n")

Summary:



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Multi-head attention The above visualization shows one attention mechanism within the model   Because the attention heads do not share parameters, each head learns a unique attention pattern  Through this repeated composition of word embeddings, BERT is able to form very rich representations as it gets to the deepest layers of the model
Word count: 52 



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


#Web Scrapping

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_website(url, delay=2):
  """
  Scrapes a website and attempts to capture more content than just the first section/paragraph.

  Args:
      url: The URL of the website to scrape.
      delay: Delay (in seconds) between requests to avoid overloading the server.

  Returns:
      A string containing the scraped content.
  """
  headers = {'User-Agent': 'Mozilla/5.0'}  # Mimic a browser to avoid blocking
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find all paragraphs
  paragraphs = soup.find_all('p')

  # Combine paragraph text along with headings (optional)
  content = ""
  for element in soup.find_all(['p', 'h1', 'h2', 'h3']):
    content += element.text.strip() + "\n"  # Add newline for readability

  # Optionally, consider using libraries like Selenium for dynamic content
  # or infinite scrolling websites (not covered in this example)

  time.sleep(delay)  # Delay to be polite to the server
  return content

# Example usage
url = "https://www.bbc.com/"  # Replace with the actual website URL
scraped_content = scrape_website(url)
print(scraped_content)

US House approves crucial $61bn war aid for Ukraine
The vote will be closely watched in Kyiv, which has warned of an urgent need for more military support.
The 'Robin Hood' TikToker taking on landlords
Jordan van den Berg's vigilante-style approach has won him fans and critics amid a housing crisis.
US House approves crucial $61bn war aid for Ukraine
The vote will be closely watched in Kyiv, which has warned of an urgent need for more military support.
The 'Robin Hood' TikToker taking on landlords
Jordan van den Berg's vigilante-style approach has won him fans and critics amid a housing crisis.
US aid could give Ukraine the weapons it needs to slow Russia’s advance
The $61bn (£49bn) aid package keeps Ukraine in the fight but it's no silver bullet, BBC's James Waterhouse writes from Kyiv.
What the world makes of Trump going on trial
Pictures of Donald Trump in a New York courtroom made headlines globally. Here's a flavour of what was said.
US migrant crisis shifts from Texas to Californ

In [ ]:
# Print the tokenized text
print("Tokenized text:\n")
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)
print("\n")

print("Summary:\n")
print(summarize(text))
print("Word count:", count_words(summarize(text)), "\n")

Tokenized text:

['us', 'house', 'approve', '##s', 'crucial', '$', '61', '##bn', 'war', 'aid', 'for', 'ukraine', 'the', 'vote', 'will', 'be', 'closely', 'watched', 'in', 'kyiv', ',', 'which', 'has', 'warned', 'of', 'an', 'urgent', 'need', 'for', 'more', 'military', 'support', '.', 'the', "'", 'robin', 'hood', "'", 'ti', '##kt', '##oke', '##r', 'taking', 'on', 'landlord', '##s', 'jordan', 'van', 'den', 'berg', "'", 's', 'vi', '##gil', '##ante', '-', 'style', 'approach', 'has', 'won', 'him', 'fans', 'and', 'critics', 'amid', 'a', 'housing', 'crisis', '.', 'us', 'house', 'approve', '##s', 'crucial', '$', '61', '##bn', 'war', 'aid', 'for', 'ukraine', 'the', 'vote', 'will', 'be', 'closely', 'watched', 'in', 'kyiv', ',', 'which', 'has', 'warned', 'of', 'an', 'urgent', 'need', 'for', 'more', 'military', 'support', '.', 'the', "'", 'robin', 'hood', "'", 'ti', '##kt', '##oke', '##r', 'taking', 'on', 'landlord', '##s', 'jordan', 'van', 'den', 'berg', "'", 's', 'vi', '##gil', '##ante', '-', 'styl

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


US House approves crucial $61bn war aid for Ukraine
The vote will be closely watched in Kyiv, which has warned of an urgent need for more military support 
How to make better New Year's resolutions
Have you 'broken' any News Year's resolutions yet? If so, you're not alone - on average 80% of resolutions fail  Here's the truth about the equation behind it 
Sport
Premier League: Liverpool extend lead at Fulham - radio and text
Solo breakaway wins Liege-Bastogne-Liege for Pogacar
Man Utd scrape past Coventry to reach FA Cup final
'We're back to where we want to be' - Clement
London Marathon to Wembley - Ratcliffe's busy Sunday
Women's Six Nations: Final round TV coverage & fixtures
Travel
Where to find the best cheesesteaks in Philly
World of Wonder
The turtle video that sparked a plastic straw revolution
Bloody and in pain, when a plastic straw became lodged in the nostril of a turtle, a video showing its removal shook the world – sparking a movement to rid the world of plastic straws  B

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


#sequence classification

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
import torch

# Load pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare your dataset and dataloaders for fine-tuning
# (Note: You need to provide your own dataset and dataloaders)

# Define optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=5e-5)

# Fine-tune the BERT model
model.train()
num_epochs = 3
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate the fine-tuned model
model.eval()
# Run evaluation on a separate test set

# Save the fine-tuned model for future use
model.save_pretrained('fine_tuned_bert_model')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


NameError: name 'dataloader' is not defined